In [2]:
from langchain.document_loaders import PyPDFLoader

from langchain.vectorstores import Chroma

from langchain.chat_models import ChatOpenAI

from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from dotenv import load_dotenv
import openai
import os

In [2]:
client = openai.OpenAI()

# Set your OpenAI API key

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
loader = PyPDFLoader("sample_data/Personalrecht.pdf")

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

pages = loader.load_and_split(text_splitter)

In [6]:
directory = 'index_store'

vector_index = Chroma.from_documents(pages, OpenAIEmbeddings(), persist_directory=directory)

vector_index.persist() # actually the Chroma client automatically persists the indexes when it is disposed - however better save then sorry :-)

In [7]:
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k":3})

# create the chain for allowing us to chat with the document

qa_interface = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

c:\Users\spunk\anaconda3\envs\ChatTest\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
result = qa_interface("In welchem Artikel werden Überstunden geregelt?")
print(result['result'])


Überstunden werden im Artikel 173 des vorliegenden Textes geregelt.
